# Training Example utilising hugging-face datasets

## Loading Raw Dataset (cifar100 example)

In [1]:
import torchvision
from datasets import load_dataset
from torchvision import transforms

cifar100 = load_dataset('cifar100')

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR100(root='../data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR100(root='../data', train=False, download=True, transform=transform)

/Users/michal/mambaforge/envs/kiss/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
Files already downloaded and verified


## Preparing DataLoaders

In [2]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

### Prepare Model & other

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50

model = resnet50(num_classes=100)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

device = torch.device("mps")
model.to(device);

In [4]:
from tqdm import tqdm

num_epochs = 2

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    with tqdm(total=len(train_loader), desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch") as pbar:
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.update(1)
            pbar.set_postfix(loss=f"{running_loss / len(train_loader):.4f}")

model.eval()
correct = 0
total = 0

with torch.no_grad():
    with tqdm(total=len(test_loader), desc="Testing", unit="batch") as pbar:
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            pbar.update(1)

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Testing: 100%|██████████| 157/157 [00:28<00:00,  5.55batch/s]

Test Accuracy: 12.85%
